In [1]:
import duckdb
import polars as pl
import pathlib

In [2]:
BASE_DATA_PATH: pathlib.Path = pathlib.Path().absolute().parent.parent / "data" / "raw"

CLIENTS_PATH: pathlib.Path = BASE_DATA_PATH / "clients"
ELECTRICITY_PATH: pathlib.Path = BASE_DATA_PATH / "electricity"
GAS_PATH: pathlib.Path = BASE_DATA_PATH / "gas"
HISTORICAL_WEATHER_PATH: pathlib.Path = BASE_DATA_PATH / "historical_weather"
WEATHER_FORECAST_PATH: pathlib.Path = BASE_DATA_PATH / "weather_forecast"
TRAIN_PATH: pathlib.Path = BASE_DATA_PATH / "train"
WEATHER_STATION_COUNTY_PATH: pathlib.Path = BASE_DATA_PATH / "weather_station_to_county_mapping.csv"
COUNTY_MAP_PATH: pathlib.Path = BASE_DATA_PATH / "county_id_to_name_map.json"

In [3]:
connection = duckdb.connect('localhost')

In [4]:
def load_train_duckdb(train_dir: pathlib.Path) -> duckdb.DuckDBPyRelation:
    return connection.sql(f"""
    SELECT
        datetime::TIMESTAMP as datetime,
        is_business::BOOLEAN as is_business,
        product_type::TINYINT as product_type,
        target::FLOAT as target,
        is_consumption::TINYINT as is_consumption,
        county::TINYINT as county,
        data_block_id::SMALLINT as data_block_id,
        date_part('year', datetime::TIMESTAMP)::SMALLINT as year,
        date_part('month', datetime::TIMESTAMP)::TINYINT as month,
        date_part('day', datetime::TIMESTAMP)::TINYINT as day,
        datetime::DATE as date,
        dayname(datetime::TIMESTAMP) as weekday,
        date_part('hour', datetime::TIMESTAMP)::tinyint as hour,
        date_part('dayofyear', datetime::TIMESTAMP)::SMALLINT as day_of_year
    FROM
        read_csv_auto('{train_dir}/*')
    """)

def load_clients_duckdb(clients_dir: pathlib.Path) -> duckdb.DuckDBPyRelation:
    return connection.sql(f"""
    SELECT
        date::DATE as date,
        product_type::TINYINT AS product_type,
        is_business::BOOLEAN as is_business,
        installed_capacity::INTEGER as installed_capacity,
        eic_count::SMALLINT as eic_count,
        county::TINYINT as county,
        data_block_id::SMALLINT as data_block_id,
    FROM
        read_csv_auto('{clients_dir}/*')
    """)

In [5]:
DATETIME_FORMAT: str = "%Y-%m-%d %H:%M:%S"
DATE_FORMAT: str = "%Y-%m-%d"

def load_train_polars(train_path: pathlib.Path) -> pl.LazyFrame:
    """Load the training data from a CSV file and perform data transformations.

    Args:
        train_path (pathlib.Path): The path to the training data CSV file.

    Returns:
        pl.LazyFrame: A lazy frame containing the loaded training data with transformed columns.

    Examples:
        >>> train_path = pathlib.Path("train.csv")
        >>> train_data = load_train(train_path)
        >>> # Perform operations on the loaded training data
    """
    train: pl.LazyFrame = pl.scan_csv(train_path)

    train = train.drop(["prediction_unit_id", "row_id"]).with_columns(
        pl.col("datetime").str.to_datetime(DATETIME_FORMAT),
        pl.col("is_business").cast(pl.Int8),
        pl.col("product_type").cast(pl.Int8),
        pl.col("target").cast(pl.Float32),
        pl.col("is_consumption").cast(pl.Int8),
        pl.col("county").cast(pl.Int8),
        pl.col("data_block_id").cast(pl.Int16),
    )
    return train.with_columns(
        pl.col("datetime").cast(pl.Date).alias("date"),
        pl.col("datetime").dt.year().alias("year"),
        pl.col("datetime").dt.month().alias("month"),
        pl.col("datetime").dt.day().alias("day"),
        pl.col("datetime").dt.weekday().alias("weekday"),
        pl.col("datetime").dt.ordinal_day().alias("day_of_year"),
        pl.col("datetime").dt.hour().alias("hour"),
    )

def load_clients_polars(clients_path: pathlib.Path) -> pl.LazyFrame:
    """Load the clients data from a CSV file and perform column type conversions.

    Args:
        clients_path (pathlib.Path): The path to the clients CSV file.

    Returns:
        pl.LazyFrame: A lazy frame containing the loaded clients data with converted column types.

    Examples:
        >>> clients_path = pathlib.Path("clients.csv")
        >>> clients_data = load_clients(clients_path)
        >>> # Perform operations on the loaded clients data
    """
    clients: pl.LazyFrame = pl.scan_csv(clients_path)
    return clients.with_columns(
        [
            pl.col("product_type").cast(pl.Int8),
            pl.col("county").cast(pl.Int8),
            pl.col("eic_count").cast(pl.Int16),
            pl.col("installed_capacity").cast(pl.Float32),
            pl.col("is_business").cast(pl.Int8),
            pl.col("date").str.to_date(DATE_FORMAT),
            pl.col("data_block_id").cast(pl.Int16),
        ]
    )

In [9]:
%%timeit

train_duckdb = load_train_duckdb(TRAIN_PATH)
clients_duckdb = load_clients_duckdb(CLIENTS_PATH)

connection.register("train", train_duckdb)
connection.register("clients", clients_duckdb)

connection.sql("""
SELECT
    *
FROM
    train
LEFT JOIN clients ON train.county = clients.county AND train.is_business = clients.is_business AND train.product_type = clients.product_type AND train.data_block_id = clients.data_block_id 
""")

463 ms ± 19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit

train_polars = load_train_polars(TRAIN_PATH / "train.csv")
clients_polars = load_clients_polars(CLIENTS_PATH / "client.csv")

data = train_polars.join(
    other=clients_polars,
    how="left",
    on=["county", "is_business", "product_type", "data_block_id"],
    suffix="_client",
).collect().to_pandas()

265 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
